# Tarea 1 - Inteligencia Artificial
Integrantes: Aarón Pozas Oyarce y Diego Muñoz Barra

## Introducción:

xxxxxxxxx

## Desarrollo y Resultados
        
xxxxxx

BLABLABLABLA

### Parte 1:

#### Definición de Librerías e Instalación de Dependencias

Primero, se procedió a instalar los paquetes y dependencias correspondientes utilizando **pip install**, en este caso se empleó para
las librerías de: **pandas**, **sklearn** y **pgmpy**. La librería **pandas** se especializa en el manejo y análisis de estructura de datos, **sklearn** sirve para el aprendizaje automático en estadística y probabilidad, y **pgmpy** sirve para realizar inferencias causales y probabilísticas que utilizan grafos acíclicos dirigidos (DAG) y redes bayesianas.

Para su utilización se ejecutó el siguiente comando para la instalación de las librerías mencionadas instalación:

In [1]:
!pip install pandas pgmpy scikit-learn matplotlib

Luego se realizan la definición de estas librerías en código de la siguiente forma:

In [2]:
import pandas as pd # Importación de Pandas
import networkx as nx # Importación de networkx: Sirve como complemento para el análisis de grafos o redes
from urllib.request import urlopen # Sirve para abrir y leer una url
from pgmpy.models import DiscreteBayesianNetwork # Es el modelo a utilizar para crear una red Bayesiana con datos estáticos o Discretos
from sklearn.model_selection import train_test_split # Sirve para separar un dataset para entrenarlo y testearlo
from pgmpy.estimators import GES # Importación de Modelo de aprendizaje GES 
from pgmpy.estimators import HillClimbSearch # Importación de Modelo de aprendizaje Hill Climb Search
from pgmpy.estimators import BayesianEstimator # Sirve para llenar las tablas de probabilidades de una red bayesiana
from pgmpy.inference import VariableElimination # Es un algoritmo para hacer inferencias probabilísticas
from pgmpy.estimators import BIC # Es una métrica estadística usada para evaluar modelos, puntuando y ajustando los datos
from sklearn.utils import resample # Sirve para crear muestras nuevas a partir de un conjunto de datos
import matplotlib.pyplot as plt
import numpy as np

C:\Users\diego\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


A partir de los comentarios, se especifica qué son y para qué sirve cada import de manera general, siendo los más importantes **DiscreteBayesianNetwork**, **Modelo GES**, **Modelo HillClimbSearch**, **BIC** y **VariableElimination**.

#### Carga y Filtrado de Datos

Posteriormente, se procede a extraer los datos desde un url para **adult.data**, para luego guardarlos en una variable llamada **'data'**. Este dataset contiene 15 columnas y un total de 32k filas aproximadamente. Para inicializar los datos se realizó lo siguiente:

In [3]:
# Definición de los Headers
headers = [ 
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 
    'marital-status', 'occupation', 'relationship', 'race', 'sex', 
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
    ]

# Definición del campo 'data' en base a los datos de la url, se le agregan los headers y se limpia la data
data = pd.read_csv(urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"), header=None, names=headers, skipinitialspace=True)
data = data.dropna() 

print(f"Datos originales: {data.shape}")

Datos originales: (32561, 15)


A partir de la impresión de anterior, se aprecia la tabla con todos sus atributos **(desde age hasta income)**, con sus respectivos datos o información correspondiente, generando un total de **32561 filas**.

#### División de Datos 

Luego, antes de realizar y crear las redes Bayesianas, es necesario separar la data para entrenar y testear los modelos. Para ello es pertinente generar una relación o proporción 70/30 en cuanto a los datos, es decir, **70% de datos para entrenamiento** y **30% de datos para pruebas**.

In [4]:
train_df, test_df = train_test_split(data, test_size=0.3, random_state=42)
print(f"Entrenamiento: {train_df.shape}, Prueba: {test_df.shape}")

Entrenamiento: (22792, 15), Prueba: (9769, 15)


Se emplean 2 variables, una para guardar los datos de entrenamiento y el otro para testeo. Se realiza un split de data designando un 0.3 como porcentaje para los datos de **test_df**, lo restante se designa a **train_df**. 

### Método de Aprendizaje GES:

El Primer Método elegido para emplear el aprendizaje de la red Bayesiana fue **GES** (Greedy Equivalence Search). Este se usa para descubrir la estructura de una red o grafo a partir de los datos disponibles. GES evalúa diferentes estructuras de red usando un criterio de puntuación, como BIC, y de manera iterativa añade, elimina o revierte aristas para mejorar dicha puntuación. El resultado es un grafo acíclico dirigido donde las aristas tienen dirección y no se forman ciclos, permitiendo interpretar relaciones condicionales de causalidad entre las variables sin necesidad de conocer la estructura completa previamente.

In [5]:
ges = GES(train_df)

metodo_bic = BIC(train_df) # Se define la métrica BIC para puntuar las redes según su complejidad, para no sobreajustar estos datos
modelo_ges = ges.estimate(scoring_method=metodo_bic) # Se genera el grafo acíclico dirigido a través de la ejecución de 'estimate'
print("Aristas aprendidas con GES:")
print(modelo_ges.edges())
print("\nNodos aprendidos con GES:")
print(modelo_ges.nodes())

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'workclass': 'C', 'fnlwgt': 'N', 'education': 'C', 'education-num': 'N', 'marital-status': 'C', 'occupation': 'C', 'relationship': 'C', 'race': 'C', 'sex': 'C', 'capital-gain': 'N', 'capital-loss': 'N', 'hours-per-week': 'N', 'native-country': 'C', 'income': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'workclass': 'C', 'fnlwgt': 'N', 'education': 'C', 'education-num': 'N', 'marital-status': 'C', 'occupation': 'C', 'relationship': 'C', 'race': 'C', 'sex': 'C', 'capital-gain': 'N', 'capital-loss': 'N', 'hours-per-week': 'N', 'native-country': 'C', 'income': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'workclass': 'C', 'fnlwgt': 'N', 'education': 'C', 'education-num': 'N', 'marital-status': 'C', 'occupation': 'C', 're

Aristas aprendidas con GES:
[('workclass', 'occupation'), ('education', 'education-num'), ('education', 'income'), ('marital-status', 'age'), ('occupation', 'education'), ('relationship', 'marital-status'), ('relationship', 'sex'), ('relationship', 'income'), ('relationship', 'workclass'), ('race', 'relationship'), ('sex', 'occupation'), ('sex', 'marital-status'), ('native-country', 'race'), ('income', 'capital-gain'), ('income', 'hours-per-week'), ('income', 'capital-loss')]

Nodos aprendidos con GES:
['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']


El algoritmo **GES** aprendió **15 nodos** correspondientes a las variables del dataset y detectó varias dependencias significativas entre ellas, teniendo un total de **16 aristas** conectadas entre nodos. Estas conecciones generan relaciones de causalidad entre los nodos, lo que permitirá posteriormente hacer inferencias a partir de estas.

### Método de Aprendizaje Hill Climb Search:

**Hill Climb Search** es otro de los métodos usados para aprender estructuras en redes bayesianas. Este es un algorítmo de optimización que **busca la mejor estructura posible** para la red bayesiana de acuerdo a una métrica de puntuación dada (BIC en este caso). Funciona con una estructura candidata (vacía, completamente conectada o aleatoria), para posteriormente **realizar iteraciones realizando** acciones como: **agregar, eliminar o revertir dirección de aristas**, evaluando los resultados con una función de puntuación, como BIC o K2. Este proceso **continúa hasta que no se encuentran mejoras**, obteniendo así un grafo acíclico dirigido que representa las dependencias más plausibles entre las variables según los datos. Para este caso se configuró de la siguiente manera:

In [6]:
hc = HillClimbSearch(train_df) # Se define el modelo con los datos de entrenamiento rapido

metodo_bic = BIC(train_df) # Se define la métrica BIC para puntuar las redes según su complejidad, para no sobreajustar estos datos
modelo_hc = hc.estimate(scoring_method=metodo_bic, max_indegree=2, max_iter=10) # Se genera el grafo acíclico dirigido a través de la ejecución de 'estimate'

print("Aristas aprendidas con HC:")
print(modelo_hc.edges())
print("\nNodos aprendidos con HC:")
print(modelo_hc.nodes())

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'workclass': 'C', 'fnlwgt': 'N', 'education': 'C', 'education-num': 'N', 'marital-status': 'C', 'occupation': 'C', 'relationship': 'C', 'race': 'C', 'sex': 'C', 'capital-gain': 'N', 'capital-loss': 'N', 'hours-per-week': 'N', 'native-country': 'C', 'income': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'workclass': 'C', 'fnlwgt': 'N', 'education': 'C', 'education-num': 'N', 'marital-status': 'C', 'occupation': 'C', 'relationship': 'C', 'race': 'C', 'sex': 'C', 'capital-gain': 'N', 'capital-loss': 'N', 'hours-per-week': 'N', 'native-country': 'C', 'income': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'workclass': 'C', 'fnlwgt': 'N', 'education': 'C', 'education-num': 'N', 'marital-status': 'C', 'occupation': 'C', 're

Aristas aprendidas con HC:
[('workclass', 'occupation'), ('education', 'education-num'), ('marital-status', 'age'), ('occupation', 'education'), ('relationship', 'marital-status'), ('relationship', 'sex'), ('relationship', 'income'), ('sex', 'occupation'), ('native-country', 'race'), ('income', 'capital-gain')]

Nodos aprendidos con HC:
['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']


El algoritmo **Hill Climb Search**, al igual que el método anterior, obtuvo detectó **15 nodos** que corresponden a los atributos del dataset (columnas), además de que mapeó unas **10 conecciones** de estos nodos. Todas estas conecciones podrán ser usadas posteriormente para realizar inferencias probabilísticas.

### Entrenamiento con Ambos Modelos

Luego de haber realizado el proceso de aprendizaje para ambos modelos, se empleó el el entrenamiento para el modelo de bayes con ambos modelos. Primero realizando una limpieza en cuanto a las aristas y nodos entregados por ambos modelos, ya que pueden existir relaciones que se repetían (ciclos de dependencia) y existían nodos que no se relacionaban con ninguna variable dentro de los modelos, por lo que antes de definir y entrenar el modelo era pertinente validar y filtrar estos parámetros.

#### GES:

In [7]:
# Limpia las conexiones eliminando duplicados inversos de aristas (Si es que existe)
aristas_ges_limpias = []
for u, v in modelo_ges.edges():
    if (v, u) not in aristas_ges_limpias:
        aristas_ges_limpias.append((u, v))

# Filtra los nodos que SOLO se encuentren concectados con otros, los que no tienen relaciones se obvian
nodos_ges = set()
for u, v in aristas_ges_limpias:
    nodos_ges.add(u)
    nodos_ges.add(v)

# Crea el modelo Bayesiano Discreto con las aristas y nodos filtradas
modelo_ges_bayes = DiscreteBayesianNetwork(aristas_ges_limpias)
modelo_ges_bayes.add_nodes_from(nodos_ges)

# Entrena el modelo Bayesiano solo con las columnas o nodos de GES
modelo_ges_bayes.fit(
    train_df[list(nodos_ges)],
    estimator=BayesianEstimator,
    prior_type='BDeu',
    equivalent_sample_size=10
)

print("\nModelo bayesiano entrenado usando la estructura GES:")
print("\nAristas del modelo:", modelo_ges_bayes.edges())
print("\nNodos del modelo:", modelo_ges_bayes.nodes())

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'race': 'C', 'education': 'C', 'workclass': 'C', 'income': 'C', 'age': 'N', 'native-country': 'C', 'hours-per-week': 'N', 'education-num': 'N', 'capital-gain': 'N', 'sex': 'C', 'marital-status': 'C', 'capital-loss': 'N', 'occupation': 'C', 'relationship': 'C'}



Modelo bayesiano entrenado usando la estructura GES:

Aristas del modelo: [('workclass', 'occupation'), ('occupation', 'education'), ('education', 'education-num'), ('education', 'income'), ('income', 'capital-gain'), ('income', 'hours-per-week'), ('income', 'capital-loss'), ('marital-status', 'age'), ('relationship', 'marital-status'), ('relationship', 'sex'), ('relationship', 'income'), ('relationship', 'workclass'), ('sex', 'occupation'), ('sex', 'marital-status'), ('race', 'relationship'), ('native-country', 'race')]

Nodos del modelo: ['workclass', 'occupation', 'education', 'education-num', 'income', 'marital-status', 'age', 'relationship', 'sex', 'race', 'native-country', 'capital-gain', 'hours-per-week', 'capital-loss']


Se aprecia que el modelo se estructuró solo con lo necesario en base a GES, omitiendo atributos como **fnlwgt** que no poseían ninguna relación dentro de la red, generándose un sistema completamente limpio y filtrado en base a este método. Cabe recalcar que quedaron las mismas relaciones, ya que no se generaron ciclos de ningún tipo.

#### Hill Climb Search:

In [8]:
# Limpia las conexiones eliminando duplicados inversos de aristas (Si es que existe)
aristas_hc_limpias = []
for u, v in modelo_hc.edges():
    if (v, u) not in aristas_hc_limpias:
        aristas_hc_limpias.append((u, v))

# Filtra los nodos que SOLO se encuentren concectados con otros, los que no tienen relaciones se obvian
nodos_hc = set()
for u, v in modelo_hc.edges():
    nodos_hc.add(u)
    nodos_hc.add(v)

# Crea el modelo Bayesiano Discreto con las aristas y nodos filtradas
modelo_hc_bayes = DiscreteBayesianNetwork(list(aristas_hc_limpias))
modelo_hc_bayes.add_nodes_from(nodos_hc) 

# Entrena el modelo Bayesiano solo con las columnas o nodos de GES
modelo_hc_bayes.fit(
    train_df[list(nodos_hc)],
    estimator=BayesianEstimator,
    prior_type='BDeu',
    equivalent_sample_size=10
)

print("\nModelo Bayesiano usando la estructura HC")
print(f"Aristas incluidas: {modelo_hc_bayes.edges()}\n")
print(f"Nodos incluidos: {modelo_hc_bayes.nodes()}")

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'race': 'C', 'education': 'C', 'workclass': 'C', 'age': 'N', 'income': 'C', 'native-country': 'C', 'education-num': 'N', 'capital-gain': 'N', 'sex': 'C', 'marital-status': 'C', 'occupation': 'C', 'relationship': 'C'}



Modelo Bayesiano usando la estructura HC
Aristas incluidas: [('workclass', 'occupation'), ('occupation', 'education'), ('education', 'education-num'), ('marital-status', 'age'), ('relationship', 'marital-status'), ('relationship', 'sex'), ('relationship', 'income'), ('sex', 'occupation'), ('income', 'capital-gain'), ('native-country', 'race')]

Nodos incluidos: ['workclass', 'occupation', 'education', 'education-num', 'marital-status', 'age', 'relationship', 'sex', 'income', 'native-country', 'race', 'capital-gain']


Se aprecia que solo se rescató lo más elemental en base al modelo **Hill Climb Search**, usando el mismo proceso que GES, con la diferencia de que al existir menos relaciones entre menos cantidad de nodos, se eliminaron más nodos que con el método anterior, siendo estos: **fnlwgt**, **capital-loss** y **hours-per-week**. Cabe recalcar, que tampoco se eliminaó ninguna relación, dado que este modelo tampoco generó aristas que tuvieran dependencia mutua entre 2 variables del sistema.

### Inferencias para cada Red

Para las inferencias se realizó un proceso muy simple a partir de una función, la cual recibe el modelo entrenado, la variable que se quiere predecir o inferir y la evidencia o supuesto para calcular esa probabilidad (variables conocidas), además del nombre del modelo. Esto en código se aprecia de la siguiente manera:

In [9]:
def inferencias(modelo, variable, evidencia, nombre_modelo):
    print(f"\n{nombre_modelo}: P({variable} | {evidencia})")
    
    try:
        inferencia = VariableElimination(modelo) # Esta permite realizar inferencias exactas pasándole el modelo y la guarda en 'inferencia'
        resultado = inferencia.query([variable], evidence=evidencia) # Realiza una query con la variable y la evidencia obteniendo los %
        print(resultado)
    except Exception as e:
        print(f"Error en modelo: {e}")

print("\nINFERENCIAS:")
# Llamados a la función con los respectivos modelos y sus inferencias específicas (variable y evidencia)
print("\nPrimera Inferencia GES y HC:")
inferencias(modelo_ges_bayes, 'income', {'education': 'HS-grad', 'age': 30}, 'GES')
inferencias(modelo_hc_bayes, 'income', {'education': 'HS-grad', 'age': 30}, 'HC')

print("\nSegunda Inferencia GES y HC:")
inferencias(modelo_ges_bayes, 'income', {'age': 40, 'sex': 'Female'}, 'GES')
inferencias(modelo_hc_bayes, 'income', {'age': 40, 'sex': 'Female'}, 'HC')

print("\nTercera Inferencia GES y HC:")
inferencias(modelo_ges_bayes, 'sex', {'income': '>50K', 'age': 32}, 'GES')
inferencias(modelo_hc_bayes, 'sex', {'income': '>50K', 'age': 32}, 'HC')

print("\nCuarta Inferencia GES y HC:")
inferencias(modelo_ges_bayes, 'occupation', {'sex': 'Male', 'income': '>50K'}, 'GES')
inferencias(modelo_hc_bayes, 'occupation', {'sex': 'Male', 'income': '>50K'}, 'HC')


INFERENCIAS:

Primera Inferencia GES y HC:

GES: P(income | {'education': 'HS-grad', 'age': 30})
+---------------+---------------+
| income        |   phi(income) |
+===============+===============+
| income(<=50K) |        0.8395 |
+---------------+---------------+
| income(>50K)  |        0.1605 |
+---------------+---------------+

HC: P(income | {'education': 'HS-grad', 'age': 30})
+---------------+---------------+
| income        |   phi(income) |
+===============+===============+
| income(<=50K) |        0.7536 |
+---------------+---------------+
| income(>50K)  |        0.2464 |
+---------------+---------------+

Segunda Inferencia GES y HC:

GES: P(income | {'age': 40, 'sex': 'Female'})
+---------------+---------------+
| income        |   phi(income) |
+===============+===============+
| income(<=50K) |        0.8420 |
+---------------+---------------+
| income(>50K)  |        0.1580 |
+---------------+---------------+

HC: P(income | {'age': 40, 'sex': 'Female'})
+-----------

Al ejecutar el código se aprecian las probabilidades de las inferencias escogidas, por ejemplo, para el caso del modelo GES de la segunda inferencia, se obtuvo que **la probabilidad de una mujer de 40 años de ganar más de 50K dólares al año es de un 15,8%**, a diferencia de la mayoría restante que gana mucho menos de eso. Para ambos modelos se realizaron las mismas inferencias, para además poder compararlos entre ellos, no teniendo tanta diferencia entre uno y otro.

### Validación de Inferencias a partir de los datos de Prueba

Para validar estas inferencias es crucial realizar una comparación con los datos de prueba (30% de los datos originales). Para esto se diseñó una función llamada **'validar_inferencia'**, a la cual, además de pasarle el modelo, variable, evidencia y nombre del modelo, recibe el **test_df**. Esta imprime los valores de probabilidad real de test_df y la diferencia con las probabilidades del modelo en sí, para apreciar qué tan acertado está.

In [10]:
print("=" * 60)
print("VALIDACIÓN SIMPLE CON 30% DE DATOS DE PRUEBA")
print("=" * 60)

def validar_inferencia(modelo, test_df, variable, evidencia, nombre_modelo):
    print(f"\n{nombre_modelo}: P({variable} | {evidencia})")

    # Esta parte realiza lo mismo que la función anterior
    try:
        inferencia = VariableElimination(modelo)
        resultado = inferencia.query([variable], evidence=evidencia)
        print("Predicción del modelo:")
        print(resultado)
    except Exception as e:
        print(f"Error en modelo: {e}")
        return
    # Este ciclo recorre las evidencias dadas y filtra solo por los atributos que se entregaron, con su respectivo valor además
    filtro = True
    for key, value in evidencia.items(): # Rescata el atributo y su valor (EJ: 'sex': 'Male')
        if key in test_df.columns: # Si ese atributo está en test.df entra al if
            filtro = filtro & (test_df[key] == value) # Filtra por ese atributo con ese valor en específico
    
    datos_filtrados = test_df[filtro] # Se pasan los valores filtrados a test_df para que solo regrese esos valores a 'datos_filtrados'
    
    prob_real = datos_filtrados[variable].value_counts(normalize=True) # Devuelve la probabilidad real usando los filtos en base a la variable
    print("\nProbabilidad real en test_df:")
    for valor, prob in prob_real.items():
        print(f"   {variable}({valor}): {prob:.4f}")

    # Imrpime la diferencia entre la probabilidad predicha del modelo y la real (prueba) de test_df
    print("\nDiferencias:")
    for i, estado in enumerate(resultado.state_names[variable]):
        predicho = resultado.values[i]
        real = prob_real.get(estado, 0)
        error = abs(predicho - real)
        print(f"   {variable}({estado}): {predicho:.4f} vs {real:.4f} | error: {error:.4f}")

# Llamados a la función de 'validar_inferencia'
print("\n" + "=" * 40)
print("PRIMERA INFERENCIA")
print("=" * 40)
validar_inferencia(modelo_ges_bayes, test_df, 'income', {'education': 'HS-grad', 'age': 30}, 'GES')
validar_inferencia(modelo_hc_bayes, test_df, 'income', {'education': 'HS-grad', 'age': 30}, 'HC')

print("\n" + "=" * 40)
print("SEGUNDA INFERENCIA")
print("=" * 40)
validar_inferencia(modelo_ges_bayes, test_df, 'income', {'age': 40, 'sex': 'Female'}, 'GES')
validar_inferencia(modelo_hc_bayes, test_df, 'income', {'age': 40, 'sex': 'Female'}, 'HC')

print("\n" + "=" * 40)
print("TERCERA INFERENCIA")
print("=" * 40)
validar_inferencia(modelo_ges_bayes, test_df, 'sex', {'income': '>50K', 'age': 32}, 'GES')
validar_inferencia(modelo_hc_bayes, test_df, 'sex', {'income': '>50K', 'age': 32}, 'HC')

print("\n" + "=" * 40)
print("CUARTA INFERENCIA")
print("=" * 40)
validar_inferencia(modelo_ges_bayes, test_df, 'occupation', {'sex': 'Male', 'income': '>50K'}, 'GES')
validar_inferencia(modelo_hc_bayes, test_df, 'occupation', {'sex': 'Male', 'income': '>50K'}, 'HC')

VALIDACIÓN SIMPLE CON 30% DE DATOS DE PRUEBA

PRIMERA INFERENCIA

GES: P(income | {'education': 'HS-grad', 'age': 30})
Predicción del modelo:
+---------------+---------------+
| income        |   phi(income) |
+===============+===============+
| income(<=50K) |        0.8395 |
+---------------+---------------+
| income(>50K)  |        0.1605 |
+---------------+---------------+

Probabilidad real en test_df:
   income(<=50K): 0.8219
   income(>50K): 0.1781

Diferencias:
   income(<=50K): 0.8395 vs 0.8219 | error: 0.0176
   income(>50K): 0.1605 vs 0.1781 | error: 0.0176

HC: P(income | {'education': 'HS-grad', 'age': 30})
Predicción del modelo:
+---------------+---------------+
| income        |   phi(income) |
+===============+===============+
| income(<=50K) |        0.7536 |
+---------------+---------------+
| income(>50K)  |        0.2464 |
+---------------+---------------+

Probabilidad real en test_df:
   income(<=50K): 0.8219
   income(>50K): 0.1781

Diferencias:
   income(<=50K):

Al ejecutar el código se aprecian las comparaciones entre las probabilidades del modelo y de test_df en base a las inferencias. Por ejemplo, si se aprecia la primera inferencia en base a GES, se tiene que la probabilidad de ganar más de 50K dólares es de **17.81% para test_df** y para el **modelo es de 16.05%**, dejando una diferencia de 1.76. Para este caso en particular, no hay demasiada diferencia contraponiendo ambas visiones, por lo que el entrenamiento y trato del modelo evidencia una precisión más que aceptable, si se considera un criterio de aceptación de menos del 10% de error.

## Aumento del Dataset en un 50% - Repetición del Proceso

Luego de todo lo anterior se procedió a aumentar el Dataset en un 50%, con el objetivo de comparar las probabilidades a partir de un aumento en el grueso de los datos. Para ello se siguió exactamente el mismo procedimiento, salvo algunas diferencias específicas.

### Proceso de Aumento de Data

Para aumentar los datos, se realizó el siguiente proceso:

In [11]:
n_adicional = int(len(data) * 0.5) # De la cantidad original de filas calcula el 50% (EJ: de 10000 -> 50% -> 5000)

datos_sinteticos = resample(data, n_samples=n_adicional, random_state=42, replace=True) # Crea un dataset con el aumento porcentual
data_original = data.copy()
data_original['origen'] = 'original'

datos_sinteticos = datos_sinteticos.copy()
datos_sinteticos['origen'] = 'sintetico'

data_aumentado = pd.concat([data_original, datos_sinteticos], ignore_index=True) # Crea el dataset con los datos originales + datos sintéticos

print(f"Datos originales: {data.shape}")
print(f"Datos sintéticos: {datos_sinteticos.shape}")
print(f"Dataset aumentado: {data_aumentado.shape}")
print(f"Aumento del {len(datos_sinteticos)/len(data_original)*100:.1f}%")

Datos originales: (32561, 15)
Datos sintéticos: (16280, 16)
Dataset aumentado: (48841, 16)
Aumento del 50.0%


In [12]:
data_aumentado

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,origen
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,original
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,original
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,original
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,original
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,original
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48836,25,Private,124590,HS-grad,9,Never-married,Exec-managerial,Other-relative,White,Male,0,0,40,United-States,<=50K,sintetico
48837,42,State-gov,24264,Some-college,10,Divorced,Transport-moving,Unmarried,White,Male,0,0,38,United-States,<=50K,sintetico
48838,68,Self-emp-inc,140852,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,<=50K,sintetico
48839,21,Private,131230,Some-college,10,Never-married,Sales,Own-child,White,Male,0,0,10,United-States,<=50K,sintetico


Se aprecia que el dataset final aumentó en un 50%, quedando en 48841 filas, con 16 columnas (una columna de origen). Posterior a esto, se realizó el mismo proceso anterior para el aprendizaje y entrenamiento.

#### División de Datos

In [13]:
train_amp, test_amp = train_test_split(data_aumentado, test_size=0.3, random_state=42)
print(f"Entrenamiento: {train_amp.shape}, Prueba: {test_amp.shape}")

Entrenamiento: (34188, 16), Prueba: (14653, 16)


### Método de Aprendizaje GES Ampliado

In [14]:
ges_amp = GES(train_amp)

metodo_bic_amp = BIC(train_amp) # Se define la métrica BIC para puntuar las redes según su complejidad, para no sobreajustar estos datos
modelo_ges_amp = ges_amp.estimate(scoring_method=metodo_bic_amp)
print("Aristas aprendidas con GES:")
print(modelo_ges_amp.edges())
print("\n")
print("Nodos aprendidos con GES:")
print(modelo_ges_amp.nodes())

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'workclass': 'C', 'fnlwgt': 'N', 'education': 'C', 'education-num': 'N', 'marital-status': 'C', 'occupation': 'C', 'relationship': 'C', 'race': 'C', 'sex': 'C', 'capital-gain': 'N', 'capital-loss': 'N', 'hours-per-week': 'N', 'native-country': 'C', 'income': 'C', 'origen': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'workclass': 'C', 'fnlwgt': 'N', 'education': 'C', 'education-num': 'N', 'marital-status': 'C', 'occupation': 'C', 'relationship': 'C', 'race': 'C', 'sex': 'C', 'capital-gain': 'N', 'capital-loss': 'N', 'hours-per-week': 'N', 'native-country': 'C', 'income': 'C', 'origen': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'workclass': 'C', 'fnlwgt': 'N', 'education': 'C', 'education-num': 'N', 'marital-status

Aristas aprendidas con GES:
[('education', 'education-num'), ('education', 'occupation'), ('education', 'income'), ('marital-status', 'age'), ('occupation', 'workclass'), ('occupation', 'relationship'), ('occupation', 'sex'), ('relationship', 'marital-status'), ('relationship', 'sex'), ('relationship', 'income'), ('relationship', 'race'), ('race', 'native-country'), ('sex', 'hours-per-week'), ('sex', 'marital-status'), ('income', 'capital-gain'), ('income', 'hours-per-week'), ('income', 'capital-loss')]


Nodos aprendidos con GES:
['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income', 'origen']


### Método de Aprendizaje Hill Climb Ampliado

In [15]:
hc_amp = HillClimbSearch(train_amp) # Se define el modelo con los datos de entrenamiento rapido

metodo_bic_amp = BIC(train_amp) # Se define la métrica BIC para puntuar las redes según su complejidad, para no sobreajustar estos datos
modelo_hc_amp = hc_amp.estimate(scoring_method=metodo_bic_amp, max_indegree=2, max_iter=10) # Se genera el grafo acíclico dirigido a través de la ejecución de 'estimate'

print("Aristas aprendidas con GES:")
print(modelo_hc_amp.edges())
print("\nNodos aprendidos con GES:")
print(modelo_hc_amp.nodes())

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'workclass': 'C', 'fnlwgt': 'N', 'education': 'C', 'education-num': 'N', 'marital-status': 'C', 'occupation': 'C', 'relationship': 'C', 'race': 'C', 'sex': 'C', 'capital-gain': 'N', 'capital-loss': 'N', 'hours-per-week': 'N', 'native-country': 'C', 'income': 'C', 'origen': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'workclass': 'C', 'fnlwgt': 'N', 'education': 'C', 'education-num': 'N', 'marital-status': 'C', 'occupation': 'C', 'relationship': 'C', 'race': 'C', 'sex': 'C', 'capital-gain': 'N', 'capital-loss': 'N', 'hours-per-week': 'N', 'native-country': 'C', 'income': 'C', 'origen': 'C'}
INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'age': 'N', 'workclass': 'C', 'fnlwgt': 'N', 'education': 'C', 'education-num': 'N', 'marital-status

Aristas aprendidas con GES:
[('education', 'occupation'), ('education-num', 'education'), ('marital-status', 'age'), ('occupation', 'workclass'), ('occupation', 'relationship'), ('relationship', 'marital-status'), ('relationship', 'sex'), ('relationship', 'income'), ('race', 'native-country'), ('income', 'capital-gain')]

Nodos aprendidos con GES:
['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income', 'origen']


### Entrenamiento de Ambos Modelos (Ampliado):

#### GES:

In [16]:
# Mismo proceso que con el Dataset Original
aristas_ges_limpias_amp = []
for u, v in modelo_ges_amp.edges():
    if (v, u) not in aristas_ges_limpias_amp:
        aristas_ges_limpias_amp.append((u, v))

nodos_ges_amp = set()
for u, v in aristas_ges_limpias_amp:
    nodos_ges_amp.add(u)
    nodos_ges_amp.add(v)

modelo_ges_bayes_amp = DiscreteBayesianNetwork(aristas_ges_limpias_amp)
modelo_ges_bayes_amp.add_nodes_from(nodos_ges_amp)

modelo_ges_bayes_amp.fit(
    train_amp[list(nodos_ges_amp)],
    estimator=BayesianEstimator,
    prior_type='BDeu',
    equivalent_sample_size=10
)

print("\nModelo bayesiano entrenado usando la estructura de GES:")
print("\nAristas del modelo:", modelo_ges_bayes_amp.edges())
print("\nNodos del modelo:", modelo_ges_bayes_amp.nodes())

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'race': 'C', 'education': 'C', 'workclass': 'C', 'income': 'C', 'age': 'N', 'native-country': 'C', 'hours-per-week': 'N', 'education-num': 'N', 'capital-gain': 'N', 'sex': 'C', 'marital-status': 'C', 'capital-loss': 'N', 'occupation': 'C', 'relationship': 'C'}



Modelo bayesiano entrenado usando la estructura de GES:

Aristas del modelo: [('education', 'education-num'), ('education', 'occupation'), ('education', 'income'), ('occupation', 'workclass'), ('occupation', 'relationship'), ('occupation', 'sex'), ('income', 'capital-gain'), ('income', 'hours-per-week'), ('income', 'capital-loss'), ('marital-status', 'age'), ('relationship', 'marital-status'), ('relationship', 'sex'), ('relationship', 'income'), ('relationship', 'race'), ('sex', 'hours-per-week'), ('sex', 'marital-status'), ('race', 'native-country')]

Nodos del modelo: ['education', 'education-num', 'occupation', 'income', 'marital-status', 'age', 'workclass', 'relationship', 'sex', 'race', 'native-country', 'hours-per-week', 'capital-gain', 'capital-loss']


#### Hill Climb

In [17]:
# Mismo proceso que con el Dataset Original
aristas_hc_limpias_amp = []
for u, v in modelo_hc_amp.edges():
    if (v, u) not in aristas_hc_limpias_amp:
        aristas_hc_limpias_amp.append((u, v))

nodos_hc_amp = set()
for u, v in modelo_hc_amp.edges():
    nodos_hc_amp.add(u)
    nodos_hc_amp.add(v)

modelo_hc_bayes_amp = DiscreteBayesianNetwork(list(aristas_hc_limpias_amp))
modelo_hc_bayes_amp.add_nodes_from(nodos_hc_amp)

modelo_hc_bayes_amp.fit(
    train_amp[list(nodos_hc_amp)],
    estimator=BayesianEstimator,
    prior_type='BDeu',
    equivalent_sample_size=10
)

print("\nModelo Hill Climb entrenado - Parámetros estimados")
print(f"Aristas incluidas: {modelo_hc_bayes_amp.edges()}\n")
print(f"Nodos incluidos: {modelo_hc_bayes_amp.nodes()}")

INFO:pgmpy: Datatype (N=numerical, C=Categorical Unordered, O=Categorical Ordered) inferred from data: 
 {'race': 'C', 'education': 'C', 'workclass': 'C', 'age': 'N', 'income': 'C', 'native-country': 'C', 'education-num': 'N', 'capital-gain': 'N', 'sex': 'C', 'marital-status': 'C', 'occupation': 'C', 'relationship': 'C'}



Modelo Hill Climb entrenado - Parámetros estimados
Aristas incluidas: [('education', 'occupation'), ('occupation', 'workclass'), ('occupation', 'relationship'), ('education-num', 'education'), ('marital-status', 'age'), ('relationship', 'marital-status'), ('relationship', 'sex'), ('relationship', 'income'), ('income', 'capital-gain'), ('race', 'native-country')]

Nodos incluidos: ['education', 'occupation', 'education-num', 'marital-status', 'age', 'workclass', 'relationship', 'sex', 'income', 'race', 'native-country', 'capital-gain']


### Inferencia de para Cada Red (Ampliado):

In [18]:
print("\nINFERENCIAS:")
# Usamos variables que sí están en el modelo GES
print("\nPrimera Inferencia GES y HC:")
inferencias(modelo_ges_bayes_amp, 'income', {'education': 'HS-grad', 'age': 30}, 'GES')
inferencias(modelo_hc_bayes_amp, 'income', {'education': 'HS-grad', 'age': 30}, 'HC')

print("\nSegunda Inferencia GES y HC:")
inferencias(modelo_ges_bayes_amp, 'income', {'age': 40, 'sex': 'Female'}, 'GES')
inferencias(modelo_hc_bayes_amp, 'income', {'age': 40, 'sex': 'Female'}, 'HC')

print("\nTercera Inferencia GES y HC:")
inferencias(modelo_ges_bayes_amp, 'sex', {'income': '>50K', 'age': 32}, 'GES')
inferencias(modelo_hc_bayes_amp, 'sex', {'income': '>50K', 'age': 32}, 'HC')

print("\nCuarta Inferencia GES y HC:")
inferencias(modelo_ges_bayes_amp, 'occupation', {'sex': 'Male', 'income': '>50K'}, 'GES')
inferencias(modelo_hc_bayes_amp, 'occupation', {'sex': 'Male', 'income': '>50K'}, 'HC')


INFERENCIAS:

Primera Inferencia GES y HC:

GES: P(income | {'education': 'HS-grad', 'age': 30})
+---------------+---------------+
| income        |   phi(income) |
+===============+===============+
| income(<=50K) |        0.8433 |
+---------------+---------------+
| income(>50K)  |        0.1567 |
+---------------+---------------+

HC: P(income | {'education': 'HS-grad', 'age': 30})
+---------------+---------------+
| income        |   phi(income) |
+===============+===============+
| income(<=50K) |        0.7662 |
+---------------+---------------+
| income(>50K)  |        0.2338 |
+---------------+---------------+

Segunda Inferencia GES y HC:

GES: P(income | {'age': 40, 'sex': 'Female'})
+---------------+---------------+
| income        |   phi(income) |
+===============+===============+
| income(<=50K) |        0.8505 |
+---------------+---------------+
| income(>50K)  |        0.1495 |
+---------------+---------------+

HC: P(income | {'age': 40, 'sex': 'Female'})
+-----------

### Validación de Interferencias a partir de datos de Prueba

Para validar los datos, se realizaron con **test_df**, esto debido a que este conjunto contiene únicamente **datos originales que no fueron usados durante el entrenamiento**, lo que permite evaluar de manera objetiva el desempeño del modelo y medir su capacidad de generalización a información nueva. Al usar el test set original, se evita la sobreestimación de la precisión que podría ocurrir si se evaluara con los datos sintéticos o aumentados, ya que estos últimos podrían reflejar patrones que el modelo ya ha visto durante el entrenamiento.

In [19]:
print("=" * 60)
print("VALIDACIÓN SIMPLE CON 30% DE DATOS DE PRUEBA")
print("=" * 60)

# Validación de Inferencias
print("\n" + "=" * 40)
print("PRIMERA INFERENCIA")
print("=" * 40)
validar_inferencia(modelo_ges_bayes_amp, test_df, 'income', {'education': 'HS-grad', 'age': 30}, 'GES')
validar_inferencia(modelo_hc_bayes_amp, test_df, 'income', {'education': 'HS-grad', 'age': 30}, 'HC')

print("\n" + "=" * 40)
print("SEGUNDA INFERENCIA")
print("=" * 40)
validar_inferencia(modelo_ges_bayes_amp, test_df, 'income', {'age': 40, 'sex': 'Female'}, 'GES')
validar_inferencia(modelo_hc_bayes_amp, test_df, 'income', {'age': 40, 'sex': 'Female'}, 'HC')

print("\n" + "=" * 40)
print("TERCERA INFERENCIA")
print("=" * 40)
validar_inferencia(modelo_ges_bayes_amp, test_df, 'sex', {'income': '>50K', 'age': 32}, 'GES')
validar_inferencia(modelo_hc_bayes_amp, test_df, 'sex', {'income': '>50K', 'age': 32}, 'HC')

print("\n" + "=" * 40)
print("CUARTA INFERENCIA")
print("=" * 40)
validar_inferencia(modelo_ges_bayes_amp, test_df, 'occupation', {'sex': 'Male', 'income': '>50K'}, 'GES')
validar_inferencia(modelo_hc_bayes_amp, test_df, 'occupation', {'sex': 'Male', 'income': '>50K'}, 'HC')

VALIDACIÓN SIMPLE CON 30% DE DATOS DE PRUEBA

PRIMERA INFERENCIA

GES: P(income | {'education': 'HS-grad', 'age': 30})
Predicción del modelo:
+---------------+---------------+
| income        |   phi(income) |
+===============+===============+
| income(<=50K) |        0.8433 |
+---------------+---------------+
| income(>50K)  |        0.1567 |
+---------------+---------------+

Probabilidad real en test_df:
   income(<=50K): 0.8219
   income(>50K): 0.1781

Diferencias:
   income(<=50K): 0.8433 vs 0.8219 | error: 0.0213
   income(>50K): 0.1567 vs 0.1781 | error: 0.0213

HC: P(income | {'education': 'HS-grad', 'age': 30})
Predicción del modelo:
+---------------+---------------+
| income        |   phi(income) |
+===============+===============+
| income(<=50K) |        0.7662 |
+---------------+---------------+
| income(>50K)  |        0.2338 |
+---------------+---------------+

Probabilidad real en test_df:
   income(<=50K): 0.8219
   income(>50K): 0.1781

Diferencias:
   income(<=50K):

Al ejecutar el código se aprecian las comparaciones entre las probabilidades del modelo aumentado y de test_df en base a las inferencias. Por ejemplo, si se aprecia la primera inferencia en base a GES, se tiene que la probabilidad de ganar más de 50K dólares es de **17.81% para test_df** (probabilidad original) y para el **modelo aumentado es de 15.67%**, dejando una diferencia de 2.13. En comparación al modelo anterior, se puede ver que la diferencia aumentó en relación al original, esto puede deberse a la **introducción de datos sintéticos que, aunque generan mayor diversidad y ayudan al modelo a aprender patrones más generales, no representan exactamente la distribución original del conjunto de test**. Es decir, al agregar datos replicados o ligeramente modificados, algunas probabilidades de ciertas combinaciones de variables pueden desplazarse, generando pequeñas diferencias en la estimación de las probabilidades condicionales. Esto es normal en métodos de aumento de datos y sirve principalmente para mejorar la robustez del modelo frente a variaciones en los datos, aunque pueda aumentar temporalmente la discrepancia con el test original.

### Comparación entre código Original y Aumentado

Finalmente, se realizó una comparación entre ambos códigos: original y aumentado. Esto con el objetivo de vaticinar la eficiencia probabilística entre ambos. Para ello se codíficó lo siguiente:

In [20]:
def comparacion(modelo_orig, modelo_aum, test_df, variable, evidencia, nombre): # Función para comparar predicciones de ambos modelos
    print(f"\n{nombre}: P({variable} | {evidencia})")
    print("-" * 50)
    
    # Se filtran las evidencias para cada modelo según las variables que realmente existen en el modelo. Evita errores si no hay nodos presentes
    evidencia_orig = {k: v for k, v in evidencia.items() if k in modelo_orig.nodes()}
    evidencia_aum = {k: v for k, v in evidencia.items() if k in modelo_aum.nodes()}
    
    # Se calculan las probabilidades según el modelo 
    def get_probs(modelo, evidence_dict):
        try:
            inferencia = VariableElimination(modelo)
            resultado = inferencia.query([variable], evidence=evidence_dict)
            return {estado: prob for estado, prob in zip(resultado.state_names[variable], resultado.values)}
        except Exception as e:
            print(f"Error: {e}")
            return None
    
    # Obtiene las predicciones para el modelo original y aumentado
    probs_orig = get_probs(modelo_orig, evidencia_orig)
    probs_aum = get_probs(modelo_aum, evidencia_aum)
    
    # Se realiza el proceso para obtener las probabilidades reales con test_df
    filtro = True
    for key, value in evidencia.items():
        if key in test_df.columns:
            filtro &= (test_df[key] == value)
    
    datos_filtrados = test_df[filtro]
    if len(datos_filtrados) > 0:
        prob_real = datos_filtrados[variable].value_counts(normalize=True).to_dict()
        
        # Muestreo de resultados
        if probs_orig:
            print("ORIGINAL:")
            for estado, prob in probs_orig.items():
                print(f"   {estado}: {prob:.4f}")
        
        if probs_aum:
            print("AUMENTADO:")
            for estado, prob in probs_aum.items():
                print(f"   {estado}: {prob:.4f}")
        
        print("REALIDAD:")
        for estado, prob in prob_real.items():
            print(f"   {estado}: {prob:.4f}")
        
        # Compara errores entre ambos modelos
        if probs_orig and probs_aum:
            print("COMPARACIÓN:")
            for estado in probs_orig.keys():
                error_orig = abs(probs_orig.get(estado, 0) - prob_real.get(estado, 0))
                error_aum = abs(probs_aum.get(estado, 0) - prob_real.get(estado, 0))
                mejora = "Mejor: Aumentado" if error_aum < error_orig else "Mejor: Original" if error_aum > error_orig else "Iguales"
                print(f"   {estado}: Orig={error_orig:.4f}, Aum={error_aum:.4f} {mejora}")
    else:
        print("Sin datos de test para esta evidencia")

# Se ejecutan las comparaciones llamando a la función
print("=" * 60)
print("COMPARACIÓN MODELOS")
print("=" * 60)

# GES
comparacion(modelo_ges_bayes, modelo_ges_bayes_amp, test_df, 'income', {'education': 'HS-grad', 'age': 30}, 'GES')
comparacion(modelo_ges_bayes, modelo_ges_bayes_amp, test_df, 'income', {'age': 40, 'sex': 'Female'}, 'GES')
comparacion(modelo_ges_bayes, modelo_ges_bayes_amp, test_df, 'sex', {'income': '>50K', 'age': 32}, 'GES')
comparacion(modelo_ges_bayes, modelo_ges_bayes_amp, test_df, 'occupation', {'sex': 'Male', 'income': '>50K'}, 'GES')

# HC  
comparacion(modelo_hc_bayes, modelo_hc_bayes_amp, test_df, 'income', {'education': 'HS-grad', 'age': 30}, 'HC')
comparacion(modelo_hc_bayes, modelo_hc_bayes_amp, test_df, 'income', {'age': 40, 'sex': 'Female'}, 'HC')
comparacion(modelo_hc_bayes, modelo_hc_bayes_amp, test_df, 'sex', {'income': '>50K', 'age': 32}, 'HC')
comparacion(modelo_hc_bayes, modelo_hc_bayes_amp, test_df, 'occupation', {'sex': 'Male', 'income': '>50K'}, 'HC')

COMPARACIÓN MODELOS

GES: P(income | {'education': 'HS-grad', 'age': 30})
--------------------------------------------------
ORIGINAL:
   <=50K: 0.8395
   >50K: 0.1605
AUMENTADO:
   <=50K: 0.8433
   >50K: 0.1567
REALIDAD:
   <=50K: 0.8219
   >50K: 0.1781
COMPARACIÓN:
   <=50K: Orig=0.0176, Aum=0.0213 Mejor: Original
   >50K: Orig=0.0176, Aum=0.0213 Mejor: Original

GES: P(income | {'age': 40, 'sex': 'Female'})
--------------------------------------------------
ORIGINAL:
   <=50K: 0.8420
   >50K: 0.1580
AUMENTADO:
   <=50K: 0.8505
   >50K: 0.1495
REALIDAD:
   <=50K: 0.7738
   >50K: 0.2262
COMPARACIÓN:
   <=50K: Orig=0.0682, Aum=0.0767 Mejor: Original
   >50K: Orig=0.0682, Aum=0.0767 Mejor: Original

GES: P(sex | {'income': '>50K', 'age': 32})
--------------------------------------------------
ORIGINAL:
   Female: 0.1801
   Male: 0.8199
AUMENTADO:
   Female: 0.1770
   Male: 0.8230
REALIDAD:
   Male: 0.8103
   Female: 0.1897
COMPARACIÓN:
   Female: Orig=0.0095, Aum=0.0127 Mejor: Original


Si analizamos los resultados, se aprecia que el modelo original tiene mejores resultados en torno a las probabilidades de **test_df**, siendo el modelo ampliado el que tiene un delta  más grande en relación a su cercanía con los valores de **test_df**, lo que implica que el modelo original es más preciso. Sin embargo, el ampliado no es necesariamente peor, ya que al tener mayor cantidad de datos repetidos tiene una distribución más suavizada, lo que puede favorecer a generalizar un poco más el dataset. 

Esto significa que, aunque el modelo aumentado muestra diferencias ligeramente mayores con respecto a los datos de prueba en ciertas evidencias puntuales, su capacidad para capturar patrones generales puede ser mejor cuando se evalúa sobre combinaciones de variables menos frecuentes o sobre todo el conjunto de datos. En otras palabras, el suavizado que introduce el aumento de datos ayuda a reducir el sobreajuste que podría presentar el modelo entrenado únicamente con el dataset original, haciendo que sus predicciones sean más estables en escenarios diversos.